In [1]:
import os
import gc
import time
import pandas as pd
import numpy as np

import geopandas as gpd
import rioxarray as rioxr
import rasterio

import data_sampling_workflow.sample_rasters as sr
from rasterio.crs import CRS

import planetary_computer as pc

from skimage.feature import graycomatrix, graycoprops
import raster_to_features as rf

In [2]:
# ***************************************************
# ************* NOTEBOOK VARIABLES ******************

window_r = 1  #in pixels

distances = [1]
angles = [np.pi/2]
props = ['contrast']#, 'correlation']

# ***************************************************
# ***************************************************

In [3]:
itemids = pd.read_csv(sr.path_to_aoi_itemids_csv())
itemids = itemids[ (itemids.aoi_name == 'point_conception') & (itemids.year == 2020)]
itemid = itemids.itemid[0]
raster = rf.rioxr_from_itemid(itemid)
#band, y_len, x_len = raster.shape
raster

<xarray.DataArray (band: 4, y: 12470, x: 10520)>
[524737600 values with dtype=uint8]
Coordinates:
  * band         (band) int64 1 2 3 4
  * x            (x) float64 7.293e+05 7.293e+05 ... 7.356e+05 7.356e+05
  * y            (y) float64 3.821e+06 3.821e+06 ... 3.813e+06 3.813e+06
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:             Area
    TIFFTAG_IMAGEDESCRIPTION:  OrthoVista
    TIFFTAG_RESOLUTIONUNIT:    1 (unitless)
    TIFFTAG_SOFTWARE:          Trimble Germany GmbH
    TIFFTAG_XRESOLUTION:       1
    TIFFTAG_YRESOLUTION:       1
    _FillValue:                0
    scale_factor:              1.0
    add_offset:                0.0
    datetime:                  2020-06-07 00:00:00+00:00

In [4]:
y_len = 100
x_len = 100
contrast = np.zeros((y_len,x_len))
band_n = 0

In [5]:
t0 = time.time()

for y in range(y_len):
    for x in range(x_len):
        window = raster[:, y-window_r:y+window_r+1, x-window_r:x+window_r+1]      # MAKE THIS A DIRECT COMPARISON BEFORE MAKING THE WINDOW OR START x,y AT RIGHT INDICES
        if (window.shape[1] != 2*window_r+1) | (window.shape[2] != 2*window_r+1):
            contrast[y,x] = np.nan
        else:
            glcm = graycomatrix(window[band_n], distances=distances, angles=angles)
            contrast[y,x] = graycoprops(glcm, 'contrast')[0,0]
            
print(time.time() - t0)

25.489594221115112


In [14]:
# The value P[i,j,d,theta] is the number of times that gray-level j occurs at a distance d and at an angle theta from gray-level i.
print(glcm.shape)
print(np.unique(glcm[:,:,0,0], return_counts=True))

(256, 256, 1, 1)
(array([0, 1], dtype=uint32), array([65530,     6]))


In [16]:
tp = [1,2,3,4]
fp = [1,2,3,4]

In [18]:
np.column_stack((tp[1:], tp[:-1]))

array([[2, 1],
       [3, 2],
       [4, 3]])

In [129]:
def L_out_R_in(raster, x,y, prev_glcm, window_r):
#    x,y = column and row in raster of center of window over which prev_glcm was calculated

    new = prev_glcm   # this is 4D array of shape 256, 256, 1, 1 (assuming 1 direction-N and 1 angle pi/2)
    top = y - window_r
    bott = y + window_r
        
    # get i,j occurrences in Lcolumn... 
    Lx = x - window_r
    pairs = np.column_stack((raster[top+1:bott+1 ,Lx], raster[top:bott ,Lx]))
    u_pairs, counts = np.unique(pairs, return_counts=True, axis=0)
    # ... and substract them 
    for pair, count in zip(u_pairs, counts):
        new[pair[0],pair[1],0,0] -= count
        
    # get i,j occurrences in Rcolumn...     
    Rx = x + window_r+1
    pairs = np.column_stack((raster[top+1:bott+1 ,Rx], raster[top:bott ,Rx]))
    u_pairs, counts = np.unique(pairs, return_counts=True, axis=0)
    # ... and add them 
    for pair, count in zip(u_pairs, counts):
        new[pair[0],pair[1],0,0] += count

    return new
    

In [ ]:
a = np.array([0,1,2,1,2,2])
raster = np.tile(a,(6,1))
raster

array([[0, 1, 2, 1, 2, 2],
       [0, 1, 2, 1, 2, 2],
       [0, 1, 2, 1, 2, 2],
       [0, 1, 2, 1, 2, 2],
       [0, 1, 2, 1, 2, 2],
       [0, 1, 2, 1, 2, 2]])

In [123]:
window = np.tile(np.array([0,1,2],dtype='uint8'),(3,1))

In [110]:
prev_glcm = np.array([[2,0,0],[0,2,0],[0,0,2]])
prev_glcm

array([[2, 0, 0],
       [0, 2, 0],
       [0, 0, 2]])

In [131]:
x,y = 1,1
window_r = 1
#prev_glcm = np.array([[2,0,0],[0,2,0],[0,0,2]])
prev_glcm = graycomatrix(window, distances=distances, angles=angles, levels=3)
for x in range(1,4):
    prev_glcm = L_out_R_in(raster, x,y, prev_glcm, window_r)
    print(prev_glcm[:,:,0,0], '\n')

[[0 0 0]
 [0 4 0]
 [0 0 2]] 

[[0 0 0]
 [0 2 0]
 [0 0 4]] 

[[0 0 0]
 [0 2 0]
 [0 0 4]] 



In [128]:
prev_glcm = graycomatrix(window, distances=distances, angles=angles, levels=3)
prev_glcm[:,:,0,0]

array([[2, 0, 0],
       [0, 2, 0],
       [0, 0, 2]], dtype=uint32)

In [91]:
pairs

array([[ 7,  3],
       [11,  7]])

In [96]:
for pair, count in zip(pairs, counts):
    glcm[pair[0],pair[1]] -= count
glcm

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -2.,  0.,  0.,  0.,  0.]])

In [7]:
for y in range(y_len):    
    for x in range(x_len):
        if x == 0:
            #window = raster[:, y-window_r:y+window_r+1, x-window_r:x+window_r+1]
            #this_glcm = graycomatrix(window[band_n], distances=distances, angles=angles)
            this_glcm = raster[:, y-window_r:y+window_r+1, x-window_r:x+window_r+1]
        else:
            this_glcm = L_out_R_in(raster, x, y, prev_glcm, window_r)
        contrast[y,x] = graycoprops(this_glcm, 'contrast')[0,0]
        prev_glcm = this_glcm
           

array([[         nan,          nan,          nan, ...,          nan,
                 nan,          nan],
       [         nan, 121.83333333,  34.        , ..., 165.66666667,
        126.16666667, 136.5       ],
       [         nan, 213.83333333, 126.        , ..., 221.66666667,
        198.16666667, 180.33333333],
       ...,
       [         nan,  24.        ,  41.33333333, ..., 105.83333333,
         55.33333333,  83.33333333],
       [         nan,  74.        ,  42.5       , ..., 136.33333333,
        280.66666667, 378.16666667],
       [         nan,  58.5       ,  21.        , ..., 103.66666667,
        241.83333333, 361.33333333]])

In [19]:
glcm

array([[[[0]],

        [[0]],

        [[0]],

        ...,

        [[0]],

        [[0]],

        [[0]]],


       [[[0]],

        [[0]],

        [[0]],

        ...,

        [[0]],

        [[0]],

        [[0]]],


       [[[0]],

        [[0]],

        [[0]],

        ...,

        [[0]],

        [[0]],

        [[0]]],


       ...,


       [[[0]],

        [[0]],

        [[0]],

        ...,

        [[0]],

        [[0]],

        [[0]]],


       [[[0]],

        [[0]],

        [[0]],

        ...,

        [[0]],

        [[0]],

        [[0]]],


       [[[0]],

        [[0]],

        [[0]],

        ...,

        [[0]],

        [[0]],

        [[0]]]], dtype=uint32)